<a href="https://colab.research.google.com/github/sabre-code/flower_classification/blob/main/classification_tfds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
print(tf.__version__)

2.8.2


In [3]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin = dataset_url, fname = 'flower_photos', untar = True)
data_dir = pathlib.Path(data_dir)

228827136/228813984 [==============================] - 2s 0us/step


In [4]:
image_count = len((list(data_dir.glob('*/*.jpg'))))
print("Total Images: ",image_count)

3670


In [14]:
rose = list(data_dir.glob('roses/*'))
sunflower = list(data_dir.glob('sunflowers/*'))
tulips = list(data_dir.glob('tulips/*'))
dandelion = list(data_dir.glob('dandelion/*'))
daisy = list(data_dir.glob('daisy/*'))

print("Total Roses :",len(rose))
print("Total Sunflowers :",len(sunflower))
print("Total Tulips :",len(tulips))
print("Total Dandelion :",len(dandelion))
print("Total Daisy :",len(daisy))

Total Roses : 641
Total Sunflowers : 699
Total Tulips : 799
Total Dandelion : 898
Total Daisy : 633


In [ ]:
PIL.Image.open(str(rose[0]))
PIL.Image.open(str(sunflower[0]))
PIL.Image.open(str(tulips[0]))
PIL.Image.open(str(dandelion[0]))
PIL.Image.open(str(daisy[0]))